## 뉴스 제목으로 주가 예측
1.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Conv1D, MaxPooling1D, GlobalMaxPooling1D , Input
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score
from transformers import AutoTokenizer, AutoModel
import torch
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('stock_news_data_large.csv')

In [ ]:
df

In [ ]:
def clean_text(inputString):
    text_rmv = re.sub('[+]','증가', inputString)
    text_rmv = re.sub('[-]','감소', text_rmv)
    text_rmv = re.sub('[↑▲]','상승', text_rmv)
    text_rmv = re.sub('[↓△]','하락', text_rmv)

    return text_rmv


In [ ]:
df['뉴스 제목'] = df['뉴스 제목'].apply(clean_text)

In [ ]:
import re

def preprocess_text(text):
    # 괄호와 괄호 안에 있는 내용 삭제
    text = re.sub(r'\([^()]*\)', '', text)  # () 안에 있는 내용 삭제
    text = re.sub(r'\[[^\[\]]*\]', '', text)  # [] 안에 있는 내용 삭제
    text = re.sub(r'\{[^{}]*\}', '', text)  # {} 안에 있는 내용 삭제

    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    text = pattern_others.sub('.', text)

    # 특수문자 제거
    text = re.sub(r'[^%.,+\-\w\s]', ' ', text)  # 특수문자를 공백으로 대체

    # 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나의 공백으로 대체
    text = text.strip()  # 문장 앞뒤의 공백 제거

    # 연속된 줄바꿈을 하나의 줄바꿈으로 대체
    text = re.sub(r'\n+', '\n', text)

    return text

def remove_stopwords(text, stopwords):
    words = text.split()
    words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(words)

stopwords = ['원본보기', '아이콘', 'AD', '자료', 'ADVERTISEMENT', 'Advertisement', 'viewer']

# Preprocess text
df['뉴스 제목'] = df['뉴스 제목'].apply(preprocess_text)

# Apply stopword removal
df['뉴스 제목'] = df['뉴스 제목'].apply(remove_stopwords, stopwords=stopwords)

# Display the dataframe
df.head()


In [ ]:
df

In [ ]:
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
texts = df['뉴스 제목'].tolist()

In [ ]:
embedded_news = []
for news in df['뉴스 제목']:
    # 문장 토큰화 및 패딩
    inputs = tokenizer.encode_plus(
        news,
        add_special_tokens=True,
        padding="max_length",
        max_length=300,
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.squeeze(0)
        embedded_news.append(embeddings.tolist())


In [ ]:
df['kcbert_embedding'] = embedded_news

In [ ]:
df

In [ ]:
df_result = pd.DataFrame()
for i in range(len(df)):
#     print(pd.DataFrame(list(X_train['dh_embedding_1'])[i]).T)

    df_result= pd.concat([df_result, pd.DataFrame(list(df['kcbert_embedding'])[i]).T], ignore_index = True)

In [ ]:
type(list(df_result.iloc[0,:]))

In [ ]:
df_result

In [ ]:
tensor_3d = []

for i in df.index :
    tensor_date = []
    tensor_date.append(list(df_result.iloc[i,:]))
    tensor_3d.append(tensor_date)

tensor_3d = np.array(tensor_3d)

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tensor_3d, df['주가_변동'], test_size = 0.2, random_state = 1, stratify = df['주가_변동'])

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras import layers
model = tf.keras.Sequential([
    layers.LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]),dropout=0.25),
    layers.LSTM(100, return_sequences=True,dropout=0.25),
    layers.LSTM(100, return_sequences=True,dropout=0.25),
    layers.Bidirectional(layers.LSTM(100)),
    layers.Dropout(0.25),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
training_record = model.fit(X_train, y_train,
                    epochs=30,
                    batch_size=128,
                    validation_data=(X_train, y_train),
                    shuffle=True)

In [ ]:
pred_proba = model.predict(X_test)

In [ ]:
plt.plot(pred_proba)

In [ ]:
fig, ax = plt.subplots()
plt.plot(training_record.history["loss"], label="Training Loss")
plt.plot(training_record.history["val_loss"], label="Validation Loss")
plt.legend()
fig.suptitle("Loss")
plt.show()


In [ ]:
# 2차원인 pred_proba를 1차원으로 변경해준다 (1000,1)=>(1000)
pred_proba_1d = pred_proba.reshape(-1)

# 임계치 이상이면 True 미만이면 False를 부여한다.
threshold = 0.43
pred = (pred_proba_1d >= threshold)


In [ ]:
# Compute the metrics
accuracy_test_rnn= accuracy_score(y_test, pred)
print(f'Accuracy: {accuracy_test_rnn}')
#=> Accuracy: 0.988

precision_test_rnn=precision_score(y_test, pred)
print(f'Precision = {round(precision_test_rnn,3)}')
#=> Precision = 0.99

recall_test_rnn=recall_score(y_test, pred)
print(f'Recall = {round(recall_test_rnn,3)}')
#=> Recall = 0.99

f1_test_rnn=f1_score(y_test, pred)
print(f'f1 score = {round(f1_test_rnn,3)}')